In [37]:
import apogee.tools.read as apread
import apogee.spec.plot as splot
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
%matplotlib inline
Lambda = splot.apStarWavegrid()
import csv
import sys

In [38]:
def find_nearest(array,value):
    index = (np.abs(array-value)).argmin()
    #return array[index]
    return index

In [39]:
def Equiv_Width(plate_number,twomass_id):
    
    #given a certain spec
    spec_header = apread.apStar(plate_number,twomass_id,ext=0,header=True)
    nvisits = spec_header[1]['NVISITS']
    EqW_array = []
    
    #calculate the emission line in a vacuum
    vhelio = spec_header[1]['VHELIO']

    c = 299792
    rydberg = 1.0973731568539*(10**7)
    electron = 9.10938356*(10**-31)
    nucleus = 1.672621898*(10**-27)

    fracryd = rydberg/(1+(electron/nucleus))
    vacuum = fracryd*((1./16.)-(1./121.))
    lambda_obs = 1/vacuum
    calculated_point1 = lambda_obs*(1+(-vhelio/c))
    calculated_point2 = calculated_point1*(10**10)
    
    
    for i in range(nvisits):
        spec = apread.apStar(plate_number,twomass_id,ext=1,header=False)[2 + i]
    
    
        centerline = find_nearest(Lambda,calculated_point2)
        L1 = centerline - 90
        L2 = centerline - 45
        R1 = centerline + 45
        R2 = centerline + 90
        #generic continuum lines 35 elements wide 
        lsum= np.sum(spec[L1:L2])/ len(spec[L1:L2])
        rsum = np.sum(spec[R1:R2])/len(spec[R1:R2])
        Fc= (lsum+rsum)/2
    
        EqW=0
        
        for i in range(L2,R1):
            summ=(Fc*(Lambda[i+1]-Lambda[i]))-((1./2.)*(Lambda[i+1]-Lambda[i])*(spec[i+1]+spec[i]))
            EqW = EqW + summ
        EqW = abs(EqW/Fc)
        EqW_array.append(EqW)
        

    STD = np.std(EqW_array)
    
    EqW_M = np.median(EqW_array)
    return twomass_id,plate_number,EqW_M,STD
    #print np.sort(EqW_array)
    #print 'The Equivalent Width is',EqW_M,'.'
    #print 'The Standard Deviation is',STD,'.'

In [64]:
def EqW_Function(filename,savefile):
    
    #INPUT
    x=[]
    exampleFile = open(filename)
    exampleReader = csv.reader(exampleFile)
    exampleData = list(exampleReader)
    for row in exampleData:
        x.append(Equiv_Width(row[2],row[1]))
    
    #OUTPUT
    f = open(savefile, 'wb')
    try:
        writer = csv.writer(f,delimiter='\t',lineterminator='\n\n')
        writer.writerow( ('#','2M ID', 'Plate ID', 'EqW','StD') )
        for i in range(len(x)):
            writer.writerow( (i,x[i][0], x[i][1],x[i][2],x[i][3]) )
    finally:
        f.close()

    print open(savefile, 'rt').read()

In [66]:
EqW_Function('FST CSV FILE.csv','FST TEST3.csv')

#	2M ID	Plate ID	EqW	StD

0	2M03434449+3143092	4586	7.9898370160982104	2.2593946604125996

1	2M03250943+3046215	4587	2.0698771175336788	0.63694837387889458

2	2M03295403+3120529	4587	1.1320212439900745	0.21907225864023644

3	2M06413876+0932117	4572	1.8984536264315228	0.36950878685894351

4	2M06410890+0929451	4572	3.1921949984133882	0.55159746179534608

5	2M06410715+0927294	4572	0.22753420433440119	0.70494264559097786

6	2M06405639+0935533	4572	3.7083618495714052	0.61930987601473142

7	2M06405426+0949203	4572	0.86631283116952496	0.31645245392345367

8	2M06404837+0948385	4572	1.365373392214839	0.84365395159920009

9	2M06403934+0934455	4572	0.47407172061032232	0.14044376435364542

10	2M06403665+0952032	4572	3.3178808164380511	0.74372091406306284

11	2M06402926+0926114	4572	0.27648737273880608	0.20842812482359555

12	2M06401370+0956305	4572	1.89738531218718	0.31132525334895894

13	2M03474160+3251437	4586	1.8661530609218397	0.33391013988714147

14	2M03451634+3206199	4586	2.2557351107127417	